<a href="https://colab.research.google.com/github/kihongMin/MLDL/blob/master/wide_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2


In [2]:
!git clone "https://github.com/parrotProj/proj1.git"

fatal: destination path 'proj1' already exists and is not an empty directory.


In [3]:
cd proj1

/content/proj1


In [0]:
import Resize

In [5]:
X_train,Y_train = Resize.train(160)
X_test,X_id = Resize.test(160)

input_size :  160


In [6]:
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(Y_train)

In [7]:
import keras.layers
import tensorflow as tf
from keras import backend as K
from keras.layers.core import Lambda
from keras.layers import Dropout

Using TensorFlow backend.


In [0]:
def zeropad(x):
    y = K.zeros(K.shape(x))
    return K.concatenate([x, y],-1)


def zeropad_output_shape(input_shape):
    shape = list(input_shape)
    assert len(shape) == 4
    shape[-1] *= 2
    return tuple(shape)

In [0]:
from keras import Input
from keras.models import Model
from keras.layers import BatchNormalization, Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D, Dense, Activation, Dropout
from keras.layers import Add, ZeroPadding2D, Flatten

In [0]:
def wideRes(x,channels,block_number,d_increase=False):
    name = str(channels) + str(block_number)
    shortcut = x  
    if d_increase == True:        
        x = Activation('relu')(x)
        x = Convolution2D(channels,kernel_size=(3,3),strides=(2,2), padding = "same", kernel_initializer = 'he_normal',name='first'+name)(x)        
        x = BatchNormalization()(x)
        
        shortcut = MaxPooling2D((2,2))(shortcut)
        #shortcut = keras.layers.concatenate([shortcut,K.zeros(K.shape(shortcut))])
        shortcut = Lambda(zeropad, output_shape=zeropad_output_shape)(shortcut)
    else:            
        x = Activation('relu')(x)
        x = Convolution2D(channels, kernel_size=(3,3), padding="same",kernel_initializer='he_normal',name = 'decrease'+name)(x)
        x = BatchNormalization()(x)

    x = Dropout(0.5)(x)

    x = Activation('relu')(x)
    x = Convolution2D(channels, kernel_size=(3,3), padding="same",kernel_initializer='he_normal',name='last'+name)(x)
    x = BatchNormalization()(x)
        
    x = Add()([x,shortcut])    
    return x

In [0]:
def block_group(x, channels, num_blocks):
    for i in range(num_blocks):    
        if i==0:    
            x = wideRes(x,channels,i,True)            
        else:
            x = wideRes(x,channels,i,False)    
    return x

In [0]:
def result(x):
    input_data=x
    k = 2
    n = 3

    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(32*k, (3,3), strides = (1,1),padding='same', kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(32*k, (3,3), strides = (1,1), padding='same', kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Convolution2D(32*k, (3,3), strides = (2,2), padding='same', kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)

    
    x = MaxPooling2D((3,3),padding='same',strides=(2,2))(x)
    

    shortcut=x

    x = Activation('relu')(x)
    x = Convolution2D(64*k, kernel_size=(3,3), padding="same",kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = Activation('relu')(x)
    x = Convolution2D(64*k, kernel_size=(3,3), padding="same",kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    shortcut = Lambda(zeropad, output_shape=zeropad_output_shape)(shortcut)
    
    x = Add()([x,shortcut]) 
    
    x = block_group(x, 128*k, n)    
    x = block_group(x, 256*k, n)
    x = block_group(x, 512*k, n)
    
    x = GlobalAveragePooling2D()(x)
    output_data = Dense(6,activation='softmax')(x)
    
    model = Model(input_data,output_data)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
    return model

In [13]:
inputs = Input(shape=(160,160,3),dtype='float32')
model = result(inputs)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.





In [14]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 160, 160, 3)  12          input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 160, 160, 3)  0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 160, 160, 64) 1792        activation_1[0][0]               
____________________________________________________________________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range=0.05, width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True, zoom_range=0.2,)
from sklearn.model_selection import train_test_split

X_Train, X_Val, Y_Train, Y_Val = train_test_split(X_train,Y_train, test_size=0.2)

In [0]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler

In [0]:
def lr(x):
    if x<15:
        return 0.01
    elif x< 30:
        return 0.001
    else:
        return 0.0001

callback_list = [ModelCheckpoint('best.h5',monitor='val_acc',save_best_only=True,save_weights_only=True),
                 LearningRateScheduler(lambda x : lr(x),1)]


In [0]:
train_generator = train_datagen.flow(X_Train,Y_Train,batch_size=64)
hist=model.fit_generator(train_generator,steps_per_epoch=len(X_Train)/64,validation_data=(X_Val,Y_Val),
               validation_steps=64,epochs=50,callbacks=callback_list)

Epoch 1/50

Epoch 00001: LearningRateScheduler setting learning rate to 0.01.
176/175 [==============================] - 79s 449ms/step - loss: 1.4826 - acc: 0.5082 - val_loss: 2.3278 - val_acc: 0.5109
Epoch 2/50

Epoch 00002: LearningRateScheduler setting learning rate to 0.01.
176/175 [==============================] - 76s 430ms/step - loss: 1.0392 - acc: 0.6319 - val_loss: 1.3420 - val_acc: 0.5191
Epoch 3/50

Epoch 00003: LearningRateScheduler setting learning rate to 0.01.
176/175 [==============================] - 76s 432ms/step - loss: 0.8324 - acc: 0.6904 - val_loss: 1.1015 - val_acc: 0.6003
Epoch 4/50

Epoch 00004: LearningRateScheduler setting learning rate to 0.01.
176/175 [==============================] - 76s 432ms/step - loss: 0.6913 - acc: 0.7476 - val_loss: 0.7158 - val_acc: 0.7585
Epoch 5/50

Epoch 00005: LearningRateScheduler setting learning rate to 0.01.
176/175 [==============================] - 76s 433ms/step - loss: 0.6136 - acc: 0.7774 - val_loss: 0.6796 - val_ac

In [0]:
fig,loss_ax=plt.subplots(figsize=(12,5))
acc_ax=loss_ax.twinx()

loss_ax.plot(hist.history['loss'],'indianred',marker='*',label='train loss')
loss_ax.plot(hist.history['val_loss'],'lightcoral',marker='*',label='val loss')

loss_ax.set_ylim([0, 2])


acc_ax.plot(hist.history['acc'],'forestgreen',marker='*',label='train accuary')
acc_ax.plot(hist.history['val_acc'],'olivedrab',marker='*',label='val accuary')
acc_ax.set_ylim([0, 1])


loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [0]:
from google.colab import files


model.save_weights("model_weight.h5")
files.download("model_weight.h5")

model.save('wide_model.h5')
files.download("wide_model.h5")

In [0]:
result = model.predict(X_test,batch_size=128)
result = np.argmax(result,axis=1).reshape(-1,1)

output = pd.DataFrame(X_id)
output['pred_label'] = result
output.columns = ['id','pred_label']
output = output.set_index('id')

In [0]:
output.to_csv('prediction.csv')


files.download("prediction.csv")

In [0]:
output['count']=1
output.groupby('pred_label').sum()